<a href="https://colab.research.google.com/github/clovisdanielss/Colab-Projects/blob/main/chatbot-bert/A_chatbot_using_transfer_lerning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_text transformers tensorflowjs
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from transformers import TFBertModel
from transformers import BertTokenizerFast

     |████████████████████████████████| 4.6 MB 9.5 MB/s 
     |████████████████████████████████| 4.2 MB 39.8 MB/s 
     |████████████████████████████████| 77 kB 5.5 MB/s 
     |████████████████████████████████| 511.7 MB 3.3 kB/s 
     |████████████████████████████████| 438 kB 61.3 MB/s 
     |████████████████████████████████| 1.6 MB 66.5 MB/s 
     |████████████████████████████████| 5.8 MB 49.6 MB/s 
     |████████████████████████████████| 6.6 MB 48.8 MB/s 
     |████████████████████████████████| 596 kB 69.2 MB/s 
     |████████████████████████████████| 84 kB 2.7 MB/s 
     |████████████████████████████████| 40 kB 4.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensor

### Defining our encoding layer

In [2]:
preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/2")
max_len=128

In [3]:
input_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
input_type_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_type_ids")
input_word_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")

In [5]:
encoder_output = encoder({"input_mask":input_mask, "input_type_ids":input_type_ids, "input_word_ids":input_word_ids})

In [6]:
encoder_output

{'default': <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'keras_layer_1')>,
 'encoder_outputs': [<KerasTensor: shape=(None, 128, 128) dtype=float32 (created by layer 'keras_layer_1')>,
  <KerasTensor: shape=(None, 128, 128) dtype=float32 (created by layer 'keras_layer_1')>,
  <KerasTensor: shape=(None, 128, 128) dtype=float32 (created by layer 'keras_layer_1')>,
  <KerasTensor: shape=(None, 128, 128) dtype=float32 (created by layer 'keras_layer_1')>,
  <KerasTensor: shape=(None, 128, 128) dtype=float32 (created by layer 'keras_layer_1')>,
  <KerasTensor: shape=(None, 128, 128) dtype=float32 (created by layer 'keras_layer_1')>,
  <KerasTensor: shape=(None, 128, 128) dtype=float32 (created by layer 'keras_layer_1')>,
  <KerasTensor: shape=(None, 128, 128) dtype=float32 (created by layer 'keras_layer_1')>,
  <KerasTensor: shape=(None, 128, 128) dtype=float32 (created by layer 'keras_layer_1')>,
  <KerasTensor: shape=(None, 128, 128) dtype=float32 (created by layer 'kera

In [7]:
model_encoder = tf.keras.models.Model(inputs={"input_mask":input_mask, "input_type_ids":input_type_ids, "input_word_ids":input_word_ids},
                                      outputs=encoder_output["sequence_output"])

In [8]:
model_encoder(preprocess(["hello"]))

<tf.Tensor: shape=(1, 128, 128), dtype=float32, numpy=
array([[[-0.6293698 ,  0.330106  ,  0.5830009 , ..., -1.0687764 ,
         -0.24665578, -0.41392398],
        [-0.09322161, -0.04189947,  0.18499213, ..., -2.1253293 ,
          0.532886  , -0.5445148 ],
        [-1.3539305 ,  0.7147312 ,  0.49062362, ...,  0.02443573,
          0.46058017, -0.77477443],
        ...,
        [-1.3059756 ,  0.28991982,  0.24231845, ..., -2.138464  ,
         -0.24562147, -0.507223  ],
        [-1.2509861 ,  0.3879781 ,  0.40541792, ..., -1.9272692 ,
          0.05003832, -0.92508066],
        [-0.30739072,  0.42505884,  0.09383105, ..., -1.5127901 ,
          0.45798174, -0.6818999 ]]], dtype=float32)>

In [9]:
model_encoder.treinable = False

### Defining our data

In [10]:
data = {
    "phrase":[
          "hi",
          "hi, how are you?",
          "Good morning?",
          "Good afternoon!!",
          "Good evening",
        "See you",
        "Bye",
        "till later",
        "Tomorrow we talk",
        "see you later",
    ],
    "intent":[
        "WELCOME",
        "WELCOME",
        "WELCOME",
        "WELCOME",
        "WELCOME",
        "FAREWELL",
        "FAREWELL",
        "FAREWELL",
        "FAREWELL",
        "FAREWELL",
    ]
}

data = pd.DataFrame(data)
data.head()

,phrase,intent
0,hi,WELCOME
1,"hi, how are you?",WELCOME
2,Good morning?,WELCOME
3,Good afternoon!!,WELCOME
4,Good evening,WELCOME


In [11]:
intents =  data.intent.unique()
intent_encoder = lambda intent: np.where(intents == intent)[0][0]
intent_encoder("WELCOME"), intents.shape

(0, (2,))

In [12]:
data["intent_encoded"] = data.intent.map(intent_encoder)
data.head()

,phrase,intent,intent_encoded
0,hi,WELCOME,0
1,"hi, how are you?",WELCOME,0
2,Good morning?,WELCOME,0
3,Good afternoon!!,WELCOME,0
4,Good evening,WELCOME,0


In [13]:
x = data.phrase.to_numpy()
y = data.intent_encoded.to_numpy()
x,y

(array(['hi', 'hi, how are you?', 'Good morning?', 'Good afternoon!!',
        'Good evening', 'See you', 'Bye', 'till later', 'Tomorrow we talk',
        'see you later'], dtype=object), array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1]))

### Creating the model

In [14]:
tf.random.set_seed(42)

In [15]:
layer_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, activation="relu"))(encoder_output["sequence_output"])
layer_dense = tf.keras.layers.Dense(intents.shape[0], activation="softmax")(layer_lstm)

model = tf.keras.models.Model(inputs={"input_mask":input_mask, "input_type_ids":input_type_ids, "input_word_ids":input_word_ids},
                                      outputs=layer_dense)
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

model.fit(preprocess(x),y, epochs=20, verbose=True)

Epoch 1/20
1/1 [==============================] - 13s 13s/step - loss: 35.8494 - accuracy: 0.6000
Epoch 2/20
1/1 [==============================] - 0s 381ms/step - loss: 0.3228 - accuracy: 0.9000
Epoch 3/20
1/1 [==============================] - 0s 358ms/step - loss: 3.0485 - accuracy: 0.5000
Epoch 4/20
1/1 [==============================] - 0s 383ms/step - loss: 0.3804 - accuracy: 0.9000
Epoch 5/20
1/1 [==============================] - 0s 360ms/step - loss: 0.3979 - accuracy: 0.8000
Epoch 6/20
1/1 [==============================] - 0s 353ms/step - loss: 0.4799 - accuracy: 0.6000
Epoch 7/20
1/1 [==============================] - 0s 359ms/step - loss: 0.5661 - accuracy: 0.5000
Epoch 8/20
1/1 [==============================] - 0s 403ms/step - loss: 0.6469 - accuracy: 0.5000
Epoch 9/20
1/1 [==============================] - 0s 415ms/step - loss: 0.7046 - accuracy: 0.5000
Epoch 10/20
1/1 [==============================] - 0s 378ms/step - loss: 0.7234 - accuracy: 0.6000
Epoch 11/20
1/1 [==

In [16]:
model.predict(preprocess(["Olá! Howdy ?"]))

1/1 [==============================] - 2s 2s/step


array([[0.927416, 0.072584]], dtype=float32)

In [17]:
layer_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(encoder_output["sequence_output"])
layer_dense = tf.keras.layers.Dense(intents.shape[0], activation="softmax")(layer_lstm)

model = tf.keras.models.Model(inputs={"input_mask":input_mask, "input_type_ids":input_type_ids, "input_word_ids":input_word_ids},
                                      outputs=layer_dense)
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

model.fit(preprocess(x),y, epochs=20, verbose=True)

Epoch 1/20
1/1 [==============================] - 7s 7s/step - loss: 0.8977 - accuracy: 0.5000
Epoch 2/20
1/1 [==============================] - 0s 349ms/step - loss: 0.6598 - accuracy: 0.4000
Epoch 3/20
1/1 [==============================] - 0s 359ms/step - loss: 0.5279 - accuracy: 0.8000
Epoch 4/20
1/1 [==============================] - 0s 359ms/step - loss: 0.4240 - accuracy: 0.9000
Epoch 5/20
1/1 [==============================] - 0s 349ms/step - loss: 0.3416 - accuracy: 1.0000
Epoch 6/20
1/1 [==============================] - 0s 366ms/step - loss: 0.2835 - accuracy: 1.0000
Epoch 7/20
1/1 [==============================] - 0s 406ms/step - loss: 0.2358 - accuracy: 1.0000
Epoch 8/20
1/1 [==============================] - 0s 392ms/step - loss: 0.1909 - accuracy: 1.0000
Epoch 9/20
1/1 [==============================] - 0s 388ms/step - loss: 0.1505 - accuracy: 1.0000
Epoch 10/20
1/1 [==============================] - 0s 359ms/step - loss: 0.1180 - accuracy: 1.0000
Epoch 11/20
1/1 [=====

In [18]:
model.predict(preprocess(["Olá! Howdy ?"]))

1/1 [==============================] - 2s 2s/step


array([[0.9884925, 0.0115075]], dtype=float32)

### Conclusion

When using tanh, the LSTM layer reduce the loss, so the probability difference between classes are greater. 

Unfortunely the keras layer from Tf_hub presents operations that we cannot use in tensorflowjs. 

So we wont use the above model. We will use bert, but from another library. 

Also we wont export the preprocessing layer, because this layer have operations that we wont support in tensorflowjs.

In [32]:
def create_model(max_len, classifier_layer=True):
    # Load tiny BERT model
    encoder = TFBertModel.from_pretrained(
        "google/bert_uncased_L-2_H-128_A-2", from_pt=True)

    # Setup input layer
    input_ids = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="input_ids")
    token_type_ids = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="token_type_ids")
    attention_mask = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="attention_mask")
    bert = encoder(
        input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
    )[0]

    # Make sure BERT weights stay the same during training
    bert.trainable = False

    # For python training we add a classification layer
    if classifier_layer:
        bert = tf.keras.layers.Dense(1, activation="sigmoid")(bert)


    # Put model together
    model = tf.keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[bert],
    )
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    optimizer = tf.keras.optimizers.Adam(lr=0.0001)
    model.compile(optimizer=optimizer, loss=[loss], metrics=["accuracy"])

    return model


# Model takes 128 tokens as input
MAX_LEN = 128

# Save model for TFJS
model_to_save = create_model(MAX_LEN, False)
model_to_save.save("./model")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

INFO:tensorflow:Assets written to: ./model/assets


INFO:tensorflow:Assets written to: ./model/assets


#### Just to we understand, we can check the output from the Bert model in transformers. 



In [23]:
encoder = TFBertModel.from_pretrained(
        "google/bert_uncased_L-2_H-128_A-2", from_pt=True)

    # Setup input layer
input_ids = tf.keras.layers.Input(
        shape=(128,), dtype=tf.int32, name="input_ids")
token_type_ids = tf.keras.layers.Input(
        shape=(128,), dtype=tf.int32, name="token_type_ids")
attention_mask = tf.keras.layers.Input(
        shape=(128,), dtype=tf.int32, name="attention_mask")
bert = encoder(
        input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
)
bert

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

TFBaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                                 <KerasTensor: shape=(None, 128, 128) dtype=float32 (created by layer 'tf_bert_model_4')>),
                                                ('pooler_output',
                                                 <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'tf_bert_model_4')>)])

### As you can see
The output from here is similar to the pooling_output and the sequence_output from the tf_hub bert model.

In [24]:
!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model model tfjs_model

2022-05-22 19:33:45.871704: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Writing weight file tfjs_model/model.json...


In [25]:
preprocess(["hello"])

{'input_mask': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 

Also the inputs are somewhat equal. But, we have different indexes. 
So if we map correctly, we can get the output from the sequence model.

In [33]:
input = preprocess(["hello"])
model_to_save({"input_ids":input["input_word_ids"], "token_type_ids":input["input_type_ids"], "attention_mask":input["input_mask"]})

<tf.Tensor: shape=(1, 128, 128), dtype=float32, numpy=
array([[[-0.7593664 ,  0.5279064 , -2.5522919 , ..., -0.91000545,
         -1.1436156 ,  0.30035463],
        [-1.689142  ,  0.5541231 , -1.0885587 , ..., -2.1692183 ,
         -1.7289025 ,  0.6991626 ],
        [-2.0604274 ,  0.641974  , -0.9092632 , ..., -1.609783  ,
         -0.97651625,  1.3083776 ],
        ...,
        [-0.8173223 ,  0.2380738 , -0.9260769 , ..., -1.2405076 ,
         -1.230954  ,  1.1262554 ],
        [-0.46505487,  0.27091733, -0.9593175 , ..., -1.1374489 ,
         -1.2489855 ,  1.0026721 ],
        [-0.28894883,  0.38519603, -0.9330457 , ..., -1.012916  ,
         -1.4600381 ,  0.8929069 ]]], dtype=float32)>

In [ ]:
from google.colab import files
!zip -r ./model.zip ./tfjs_model
files.download("model.zip")